<a href="https://colab.research.google.com/github/timalsinab/Bishal/blob/master/L04_Shelling_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [L04: Implementing the Shelling Model](https://docs.google.com/document/d/1NtGAKnsfwLRYMYrL74aMVMtTuQRJ1oDjvSFWlmMjpzs/edit?usp=sharing)
Written by Dr. Jan Pearce, Berea College

Complete the code by meeting all linked requirements and fixing all FIXMEs

## Your Name: Bishal Timalsina

**SUMMARY**: A brief summary description of the design and implementation, including how much your initial design plan evolved, the final result you achieved and the amount of time you spent as a programmer or programmers in accomplishing these results. This should be no more than two paragraphs.

When I first started this lab, I didn’t think the Schelling Model would be that interesting — it just sounded like “people move if they’re unhappy.” But once I got the simulation running, it actually felt alive. Watching the grid shift and seeing red and blue groups slowly form was kind of wild. I changed a lot from my original plan because things never work exactly how you imagine. I had to redo how the agents checked their neighbors and how the movement worked. Once I finally got the animation to run smoothly in Colab, it all came together.

I spent around seven hours on this, most of it just tweaking little details and fixing stuff that broke every time I changed one thing. It was frustrating, but also satisfying when everything finally worked and I could see the model stabilizing step by step.

**PERSONNEL**: A description of who you worked with and on what sections.  It is expected that all collaboration is done as pair programming together. Also, note these collaborations in the code section itself.

I coded this part by myself, but I talked with my teammates about how to structure the classes and what variables we’d need. We shared ideas at the start, but once I started coding, I mostly worked solo in Colab, testing and adjusting as I went.

**CHALLENGES**: Descriptions of the largest challenges you overcame and what made them challenging.

Getting the animation to actually move was a pain. Colab doesn’t play animations normally, so for a while I thought something was wrong with my logic when really it just wasn’t displaying. I also had to figure out a good way to measure segregation — my first few ideas gave random numbers that didn’t match what I saw. Debugging the movement logic was another long process; sometimes agents just refused to move even though they were supposed to.

**INNOVATIONS**: Any innovations that were not specifically required by the assignment. These are not required, but should be highlighted if included.

I made the model actually animate step by step instead of just showing a single snapshot, which made it way easier to see what was happening. I also added user input so you can change the grid size, tolerance, and red/blue ratio without editing code. I included two measures of segregation (edge agreement and average neighbor similarity) because I wanted the results to feel more complete.

**TESTING**: Describe how you tested this work.

I tested it mostly by running small grids over and over and watching how they behaved. I also used a few unit tests for neighbor checking and movement to make sure the math part was right. Changing the parameters was the best test, though — if the results didn’t make sense visually, I knew something was off.

**ERRORS**: A list in bulleted form of all known errors and deficiencies.
Sometimes the animation stops too early or moves really fast in Colab.

If there are too many empty cells, the model ends too quickly.

The grid edges don’t wrap around.

Randomness makes each run a little different.

**COMMENTS**: A paragraph or so of your own comments on and reactions to the Lab.

This lab felt different because it wasn’t just about getting the code to run — it was about watching something take shape on its own. It made me think about how small preferences can cause big changes in a system. Honestly, seeing the grid slowly organize itself was kind of satisfying after hours of debugging. It reminded me why I like programming — when it finally works, it just clicks.

## Import Libraries

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import itertools
import unittest
import os
from IPython.display import HTML, display

## The Schelling Model

In [5]:
EMPTY = 0
RED = 1
BLUE = 2

class City:
    def __init__(self, width, height, neighbor_mode="moore"):
        self.width = int(width)
        self.height = int(height)
        self.neighbor_mode = neighbor_mode
        self.grid = np.zeros((self.height, self.width), dtype=np.int8)

    def random_initialize(self, occupancy, red_share, rng):
        n = self.width * self.height
        n_occ = int(round(occupancy * n))
        n_red = int(round(red_share * n_occ))
        n_blue = n_occ - n_red
        cells = [RED]*n_red + [BLUE]*n_blue + [EMPTY]*(n - n_occ)
        rng.shuffle(cells)
        self.grid = np.array(cells, dtype=np.int8).reshape(self.height, self.width)

    def load_from_csv(self, path):
        try:
            arr = np.loadtxt(path, delimiter=",", dtype=np.int8)
        except ValueError:
            arr = np.loadtxt(path, dtype=np.int8)
        self.height, self.width = arr.shape
        self.grid = arr

    def neighbors_of(self, r, c):
        if self.neighbor_mode == "von_neumann":
            deltas = [(-1,0),(1,0),(0,-1),(0,1)]
        else:
            deltas = list(itertools.product([-1,0,1], repeat=2))
        out = []
        for dr, dc in deltas:
            if dr == 0 and dc == 0:
                continue
            rr, cc = r + dr, c + dc
            if 0 <= rr < self.height and 0 <= cc < self.width:
                out.append((rr, cc))
        return out

    def like_neighbor_ratio(self, r, c):
        me = self.grid[r, c]
        if me == EMPTY:
            return 0.0
        nbrs = self.neighbors_of(r, c)
        vals = [self.grid[rr, cc] for rr, cc in nbrs if self.grid[rr, cc] != EMPTY]
        if not vals:
            return 0.0
        like = sum(1 for v in vals if v == me)
        return like / len(vals)

    def edge_agreement(self):
        agree = 0
        total = 0
        for r in range(self.height):
            for c in range(self.width):
                me = self.grid[r, c]
                if me == EMPTY:
                    continue
                if c+1 < self.width and self.grid[r, c+1] != EMPTY:
                    total += 1
                    if self.grid[r, c+1] == me:
                        agree += 1
                if r+1 < self.height and self.grid[r+1, c] != EMPTY:
                    total += 1
                    if self.grid[r+1, c] == me:
                        agree += 1
        return 1.0 if total == 0 else agree / total

    def mean_like_share(self):
        shares = []
        for r in range(self.height):
            for c in range(self.width):
                if self.grid[r, c] != EMPTY:
                    shares.append(self.like_neighbor_ratio(r, c))
        return 0.0 if not shares else float(np.mean(shares))

    def empty_cells(self):
        rr, cc = np.where(self.grid == EMPTY)
        return list(zip(rr.tolist(), cc.tolist()))

    def occupied_cells(self):
        rr, cc = np.where(self.grid != EMPTY)
        return list(zip(rr.tolist(), cc.tolist()))


class Simulation:
    def __init__(self, city, tolerance, rng=None):
        self.city = city
        self.tolerance = float(tolerance)
        self.rng = rng or random.Random()
        self.moves = 0

    def is_happy(self, r, c):
        return self.city.like_neighbor_ratio(r, c) >= self.tolerance

    def step(self):
        coords = self.city.occupied_cells()
        self.rng.shuffle(coords)
        empties = self.city.empty_cells()
        self.rng.shuffle(empties)
        moved = 0
        for r, c in coords:
            if self.is_happy(r, c):
                continue
            if not empties:
                continue
            i = self.rng.randrange(len(empties))
            rr, cc = empties.pop(i)
            self.city.grid[rr, cc] = self.city.grid[r, c]
            self.city.grid[r, c] = EMPTY
            moved += 1
        self.moves += moved
        return moved

    def run(self, max_steps=200):
        for _ in range(max_steps):
            if self.step() == 0:
                break
        return self.moves


class Renderer:
    def __init__(self, city):
        self.city = city
        self.cmap = plt.matplotlib.colors.ListedColormap(["#FFFFFF", "#E74C3C", "#3498DB"])
        self.norm = plt.matplotlib.colors.BoundaryNorm([-0.5, 0.5, 1.5, 2.5], self.cmap.N)

    def animate(self, sim, frames=100, interval_ms=200):
        fig, ax = plt.subplots(figsize=(5,5))
        im = ax.imshow(self.city.grid, cmap=self.cmap, norm=self.norm)
        ax.set_xticks([]); ax.set_yticks([])
        title = ax.set_title("Schelling Model")

        def _step(frame):
            moved = sim.step()
            im.set_data(self.city.grid)
            title.set_text(f"Step {frame+1} | Moved: {moved}")
            return (im,)

        anim = animation.FuncAnimation(fig, _step, frames=frames, interval=interval_ms, blit=True)
        plt.close(fig)
        return HTML(anim.to_jshtml())


def run_experiment(width, height, occupancy_pct, red_share_pct, tolerance_pct,
                   init_method="random", csv_path=None, neighbor_mode="moore",
                   seed=42, animate=True, max_steps=200):
    rng = random.Random(seed)
    city = City(width, height, neighbor_mode=neighbor_mode)

    if init_method == "random":
        city.random_initialize(occupancy_pct/100.0, red_share_pct/100.0, rng)
    elif init_method == "file":
        if csv_path is None or not os.path.exists(csv_path):
            raise ValueError("CSV path not found.")
        city.load_from_csv(csv_path)

    init_edge = city.edge_agreement()
    init_mean = city.mean_like_share()

    sim = Simulation(city, tolerance=tolerance_pct/100.0, rng=rng)

    if animate:
        display(Renderer(city).animate(sim, frames=max_steps, interval_ms=200))
    else:
        sim.run(max_steps=max_steps)

    final_edge = city.edge_agreement()
    final_mean = city.mean_like_share()

    print("\n=== Simulation Results ===")
    print(f"Grid: {city.width} x {city.height}")
    print(f"Occupancy: {occupancy_pct:.1f}% | Red share: {red_share_pct:.1f}% | Tolerance: {tolerance_pct:.1f}%")
    print(f"Neighbor mode: {neighbor_mode}")
    print(f"Initial edge agreement: {init_edge:.3f}")
    print(f"Initial mean like-share: {init_mean:.3f}")
    print(f"Final edge agreement:   {final_edge:.3f}")
    print(f"Final mean like-share:  {final_mean:.3f}")
    print(f"Total moves: {sim.moves}")


class TestCity(unittest.TestCase):
    def test_neighbors_moore(self):
        city = City(3,3,"moore")
        self.assertEqual(len(set(city.neighbors_of(1,1))), 8)

    def test_edge_agreement(self):
        city = City(2,2)
        city.grid = np.array([[RED, RED],[BLUE, BLUE]], dtype=np.int8)
        self.assertAlmostEqual(city.edge_agreement(), 0.5, places=6)

class TestSimulation(unittest.TestCase):
    def test_move_happens(self):
        rng = random.Random(0)
        city = City(2,2)
        city.grid = np.array([[RED, EMPTY],[BLUE, EMPTY]], dtype=np.int8)
        sim = Simulation(city, tolerance=0.5, rng=rng)
        moved = sim.step()
        self.assertGreaterEqual(moved, 1)

def run_tests():
    suite = unittest.TestLoader().loadTestsFromModule(__import__(__name__))
    unittest.TextTestRunner(verbosity=2).run(suite)


def main():
    width = int(input("Enter width: "))
    height = int(input("Enter height: "))
    occupancy = float(input("Enter occupancy percentage: "))
    red_share = float(input("Enter red percentage of agents: "))
    tolerance = float(input("Enter contentedness threshold (percentage): "))
    method = input("Enter init method (random/file): ").strip().lower()
    csv_path = None
    if method == "file":
        csv_path = input("Enter CSV path: ").strip()
    run_experiment(width, height, occupancy, red_share, tolerance, init_method=method, csv_path=csv_path)

if __name__ == "__main__":
    main()

Enter width: 20
Enter height: 20
Enter occupancy percentage: 90
Enter red percentage of agents: 50
Enter contentedness threshold (percentage): 40
Enter init method (random/file): random



=== Simulation Results ===
Grid: 20 x 20
Occupancy: 90.0% | Red share: 50.0% | Tolerance: 40.0%
Neighbor mode: moore
Initial edge agreement: 0.490
Initial mean like-share: 0.475
Final edge agreement:   0.829
Final mean like-share:  0.809
Total moves: 262


## Integrity statement

Please briefly describe all references you used, all help you received and all help you gave to others in completing this assignment. Be sure to say that you got no help if you got none.

FIXME